In [34]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [35]:
house_dict = []

for page in range(16, 20):
    # Filter Out Coming Soon, Auction, Home Type: Houses, Days On Zillow: 90 Days
    linked = 'https://www.zillow.com/ca/condos/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-153.847663625,"east":-84.765632375,"south":12.777434491046959,"north":56.013075839104054},"mapZoom":4,"regionSelection":[{"regionId":9,"regionType":2}],"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"},"isSingleFamily":{"value":false}},"isListVisible":true,"isMapVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")
            
            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[0].text
                if (zestimate == "Zestimate®"):
                    zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = loop_soup.findAll('p', class_="Text-aiai24-0 sc-eHgmQL sc-kasBVs iZUTqI")[1].text
            except:
                zestimate = 'null'
#                 print('zestimate', zestimate)
                rent_zestimate = 'null'
#             print(zestimate)

            
            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(2)
            
        except AttributeError as e:
            continue

# print(house_dict)

In [36]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,HOA,House_type,House_website,Listing_type,Lot,Lot_conversion_to_sqft,Parking,Price,Rent_zestimate,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,2,2,SanDiego,"1263 Robinson Ave APT 8, San Diego, CA 92103",$343/month,Condo,https://www.zillow.com/homedetails/1263-Robins...,Condo for sale,null,null,2 spaces,"$495,000","$2,475/mo","1,024",CA,1263 Robinson Ave APT 8,1982,"$498,629",$483,92103
1,2,2,PalmSprings,"1749 E Sandalwood Dr, Palm Springs, CA 92262",$480/month,Condo,https://www.zillow.com/homedetails/1749-E-Sand...,Condo for sale,null,null,2 spaces,"$289,000","$1,650/mo","1,050",CA,1749 E Sandalwood Dr,1972,"$285,835",$275,92262
2,3,2,Irvine,"2 Cigliano Aisle, Irvine, CA 92606",$367/month,Condo,https://www.zillow.com/homedetails/2-Cigliano-...,Condo for sale,null,null,2 spaces,"$599,900","$2,990/mo","1,347",CA,2 Cigliano Aisle,1990,"$596,370",$445,92606
3,2,2,Irvine,"2212 Apricot Dr #2212, Irvine, CA 92618",$440/month,Condo,https://www.zillow.com/homedetails/2212-Aprico...,Condo for sale,null,null,No Data,"$432,000",null,916,CA,2212 Apricot Dr #2212,1979,null,$472,92618
4,2,3,SanBernardino,"4748 Woodbend Ln, San Bernardino, CA 92407",$340/month,Condo,https://www.zillow.com/homedetails/4748-Woodbe...,Condo for sale,null,null,2 spaces,"$260,000","$1,700/mo","1,792",CA,4748 Woodbend Ln,1969,"$261,906",$145,92407
5,1,1,Oakland,"4146 Manila Ave #A, Oakland, CA 94609",$325/month,Condo,https://www.zillow.com/homedetails/4146-Manila...,Condo for sale,null,null,No Data,"$749,000",null,906,CA,4146 Manila Ave #A,1918,null,$827,94609
6,1,1,SantaClara,"1883 Agnew Rd UNIT 247, Santa Clara, CA 95054",$357/month,Condo,https://www.zillow.com/homedetails/1883-Agnew-...,Condo for sale,null,null,1 space,"$619,000","$2,799/mo",870,CA,1883 Agnew Rd UNIT 247,2006,"$645,913",$711,95054
7,2,2,LosAngeles,"20235 Keswick St UNIT 307, Los Angeles, CA 91306",$390/month,Condo,https://www.zillow.com/homedetails/20235-Keswi...,Condo for sale,null,null,No Data,"$349,000","$2,300/mo","1,082",CA,20235 Keswick St UNIT 307,1982,"$353,498",$323,91306
8,1,2,Oceanside,"4614 Calle De Retiro, Oceanside, CA 92057",No Data,Condo,https://www.zillow.com/homedetails/4614-Calle-...,Condo for sale,null,null,2 spaces,"$399,000","$2,085/mo",827,CA,4614 Calle De Retiro,1979,"$363,314",$482,92057
9,1,2,LomaLinda,"1965 Coulston St APT 66, Loma Linda, CA 92354",$325/month,Condo,https://www.zillow.com/homedetails/1965-Coulst...,Condo for sale,null,null,2 spaces,"$235,000","$1,505/mo",936,CA,1965 Coulston St APT 66,1982,"$236,446",$251,92354


In [37]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2020_Condo_Data_page16-20.csv", index=False, header=True)